In [85]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

# Load data

In [86]:
data = load_breast_cancer()
X = data['data']
y = data['target'].reshape((-1, 1))

In [340]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [341]:
X_test.shape

(114, 30)

# Test with sklearn

In [342]:
y_test.shape

(114, 1)

In [90]:

mlp.fit(X_test, y_test.ravel())

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(10, 10, 10, 10, 10), learning_rate='constant',
              learning_rate_init=10, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=42, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=True, warm_start=False)

In [319]:
nb_splits = 5
skf = StratifiedKFold(n_splits=nb_splits, shuffle=True, random_state=42)
mlp = MLPClassifier((50, 50, 50, 50, 50, 50, ), activation='tanh', solver='lbfgs', 
                    learning_rate='constant', learning_rate_init=0.01, random_state=42, verbose=True)
score_train = list()
score_test = list()

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index].ravel(), y[test_index].ravel()
    mlp.fit(X_train, y_train)
    score_train.append(accuracy_score(y_train, mlp.predict(X_train)))
    score_test.append(accuracy_score(y_test, mlp.predict(X_test)))

    
print("Mean accuracy on train : {0:.1f}%".format(100 * np.mean(score_train)))
print("Standard deviation on train : {0:.1f}%".format(100 * np.std(score_train)))

print("Mean accuracy on test : {0:.1f}%".format(100 * np.mean(score_test)))
print("Standard deviation on test : {0:.1f}%".format(100 * np.std(score_test)))


Mean accuracy on train : 93.2%
Standard deviation on train : 1.3%
Mean accuracy on test : 92.4%
Standard deviation on test : 1.7%


# Forward propagation

i is the number of the layer
$$A_{i} = g_{i}(Z_{i})$$
$$Z_{i} = A_{i-1}W_{i}.T + B_{i}$$


# Backward propagation

- last layer (L)
$$dZ_{L} = A_{L} - Y$$


- other layers (i)
$$dZ_{i} = W_{i+1}^{T}dZ_{i+1} * g_{i}'(Z_{i})$$


- All layers
$$dW_{i} = \frac{1}{n} * dZ_{i}A_{i-1}^{T}$$
$$db_{i} =  mean(dZ_{i})$$




# Neural Network code

In [234]:
def sigmoid(X):
    return 1 / (1 + np.exp(-X))

def tanh_derivative(X):
    return 1 - np.tanh(X) ** 2

In [289]:
class Layer(object):
    seed = 42
    
    def __init__(self, idx, n_neurons, input_shape):
        self.idx = idx
        self.n_neurons = n_neurons
        self.W = np.random.RandomState(Layer.seed).normal(size=(n_neurons, input_shape[1]))
        self.b = np.ones((n_neurons, 1))
        self.Z = None
        self.output = None
        self.dZ = None
    
    def forward_propagation(self, input_layer, a_function):
        self.Z = np.dot(input_layer, self.W.T) + self.b.T
        self.output = a_function(self.Z)

        return self.output
    
    def backward_propagation(self, W_next, dZ_next, output_previous, derivative_a_function, learning_rate):

        self.dZ = np.dot(dZ_next, W_next) * derivative_a_function(self.Z)    
        dW = np.dot(self.dZ.T, output_previous) / self.dZ.shape[0]
        db = np.sum(self.dZ.T, axis=1, keepdims=True) / self.dZ.shape[0]
        
        self.W = self.W - learning_rate * dW
        self.b = self.b - learning_rate * db
        
        return None
        
    def backward_propagation_last_layer(self, y, output_previous, learning_rate):
        y_pred = self.output
        self.dZ = y_pred - y
        
        dW = np.dot(self.dZ.T, output_previous) / self.dZ.shape[0]
        db = np.sum(self.dZ.T, axis=1, keepdims=True) / self.dZ.shape[0]
        self.W = self.W - learning_rate * dW
        self.b = self.b - learning_rate * db
        
        return None
    

In [376]:
class NeuralNetwork(object):
    
    def __init__(self, n_layers, n_neurons, learning_rate, max_epoch):
        self.n_layers = n_layers
        self.n_neurons = n_neurons
        self.learning_rate = learning_rate
        self.max_epoch = max_epoch
        self.all_layers = list()
        
    def forward_propagation(self, X):
        input_layer = X
        
        for layer in self.all_layers[: -1]:
            input_layer = layer.forward_propagation(input_layer, np.tanh)
        output = self.all_layers[-1].forward_propagation(input_layer, sigmoid)

        return output
    
    def backward_propagation(self, X, y):
        
        for i, layer in enumerate(reversed(self.all_layers)):
            reversed_i = self.n_layers - i - 1
            
            if reversed_i > 0:
                output_previous = self.all_layers[reversed_i - 1].output
            else:
                output_previous = X
            
            if reversed_i == self.n_layers - 1:
                layer.backward_propagation_last_layer(y, output_previous, self.learning_rate)
            else:
                W_next = self.all_layers[reversed_i + 1].W
                dZ_next = self.all_layers[reversed_i + 1].dZ
                layer.backward_propagation(W_next, dZ_next, output_previous, tanh_derivative, self.learning_rate)
        
        return None
        
    def compute_loss(self, y_pred, y):
        total_loss = np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))
        return - total_loss
    
    def init_layers(self, input_shape): 
        
        for i in range(self.n_layers):
            if i == self.n_layers - 1:
                layer_i = Layer(i, 1, input_shape)
            else:
                layer_i = Layer(i, self.n_neurons, input_shape)
            input_shape = (input_shape[0], self.n_neurons)
            self.all_layers.append(layer_i)
        
        return None
    
    def fit(self, X, y):
        self.init_layers(X.shape)
        stop_tolerance = 1e-4
        stop_nb_iter = 5
            
        saved_losses = list()
        for i in range(self.max_epoch):
            y_pred = self.forward_propagation(X)
            self.backward_propagation(X, y)
            loss = self.compute_loss(y_pred, y)
            
            if saved_losses and abs(loss - saved_losses[-1]) < stop_tolerance:
                nb_iter_in_tolerance += 1
            else:
                nb_iter_in_tolerance = 0
            
            if nb_iter_in_tolerance >= stop_nb_iter:
                print("The Neural network has converged in {} epochs".format(i))
                print("The Neural network Loss is : {}".format(loss))
                break
            if i == self.max_epoch - 1:    
                print("The Neural network has not converged")
            saved_losses.append(loss)
            
    def predict(self, X):
        y_pred = np.rint(self.forward_propagation(X))
        return y_pred

In [389]:
nn = NeuralNetwork(4, 50, learning_rate=0.001, max_epoch=10000)
nn.fit(X_train, y_train)

The Neural network has converged in 1232 epochs
The Neural network Loss is : 0.26738905379209127


In [390]:
y_pred = nn.predict(X_test)
accuracy_score(y_pred, y_test)

0.9298245614035088

In [391]:
y_pred = nn.predict(X_train)
accuracy_score(y_pred, y_train)

0.9032967032967033